In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
albums,table
sqlite_sequence,table
artists,table
customers,table
employees,table
genres,table
invoices,table
invoice_items,table
media_types,table
playlists,table


In [14]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT ii.* FROM invoice_items ii
    INNER JOIN invoices i on ii.InvoiceId = i.InvoiceId
    INNER JOIN customers c on i.CustomerId = c.CustomerId
    WHERE c.country = "USA"
   )

SELECT * FROM usa_tracks_sold LIMIT 5;

 * sqlite:///chinook.db
Done.


InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
74,13,462,0.99,1
723,134,907,0.99,1
724,134,909,0.99,1
782,145,1236,0.99,1
783,145,1245,0.99,1


In [17]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT ii.* FROM invoice_items ii
    INNER JOIN invoices i on ii.InvoiceId = i.InvoiceId
    INNER JOIN customers c on i.CustomerId = c.CustomerId
    WHERE c.country = "USA"
   )

SELECT
    g.Name genres,
    count(uts.InvoiceLineId) tracks_sold,
    cast(count(uts.InvoiceLineId) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN tracks t on t.TrackId = uts.TrackId
INNER JOIN genres g on g.GenreId = t.GenreId
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


genres,tracks_sold,percentage_sold
Rock,157,0.31781376518218624
Latin,91,0.18421052631578946
Metal,64,0.12955465587044535
Alternative & Punk,50,0.10121457489878542
Jazz,22,0.044534412955465584
Blues,15,0.030364372469635626
TV Shows,14,0.02834008097165992
R&B/Soul,12,0.024291497975708502
Classical,8,0.016194331983805668
Comedy,8,0.016194331983805668


In [20]:
%%sql

WITH customer_support_rep_sales AS
    (
     SELECT
         i.CustomerId,
         c.SupportRepId,
         SUM(i.total) total
     FROM invoices i
     INNER JOIN customers c ON i.CustomerId = c.CustomerId
     GROUP BY 1,2
    )

SELECT
    e.FirstName || " " || e.LastName employees,
    e.HireDate,
    SUM(csrs.total) total_sales
FROM customer_support_rep_sales csrs
INNER JOIN employees e ON e.EmployeeId = csrs.SupportRepId
GROUP BY 1;

 * sqlite:///chinook.db
Done.


employees,HireDate,total_sales
Jane Peacock,2002-04-01 00:00:00,833.0400000000001
Margaret Park,2003-05-03 00:00:00,775.4000000000001
Steve Johnson,2003-10-17 00:00:00,720.1600000000001


In [29]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customers
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.CustomerId,
       ii.*
     FROM invoice_items ii
     INNER JOIN invoices i ON i.InvoiceId = ii.InvoiceId
     INNER JOIN customers c ON c.CustomerId = i.CustomerId
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct CustomerId) customers,
        SUM(UnitPrice) total_sales,
        SUM(UnitPrice) / count(distinct CustomerId) customer_lifetime_value,
        SUM(UnitPrice) / count(distinct InvoiceId) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,523.060000000003,5.747912087912121,40.235384615384845
Canada,8,303.9600000000013,5.427857142857166,37.99500000000016
France,5,195.1000000000003,5.574285714285723,39.02000000000006
Brazil,5,190.10000000000028,5.4314285714285795,38.02000000000005
Germany,4,156.48,5.588571428571428,39.12
United Kingdom,3,112.85999999999979,5.374285714285704,37.619999999999926
Czech Republic,2,90.23999999999997,6.445714285714283,45.11999999999998
Portugal,2,77.23999999999998,5.517142857142856,38.61999999999999
India,2,75.25999999999998,5.789230769230768,37.62999999999999
Other,15,604.3000000000037,5.755238095238131,40.28666666666691


In [34]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         ii.InvoiceId invoice_id,
         MIN(ii.TrackId) first_track_id
     FROM invoice_items ii
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoices
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.TrackId FROM tracks t
                  WHERE t.AlbumId = (
                                      SELECT t2.AlbumId FROM tracks t2
                                      WHERE t2.TrackId = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT ii2.TrackId FROM invoice_items ii2
                  WHERE ii2.InvoiceId = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT ii2.TrackId FROM invoice_items ii2
                  WHERE ii2.InvoiceId = ifs.invoice_id

                  EXCEPT 

                  SELECT t.TrackId FROM tracks t
                  WHERE t.AlbumId = (
                                      SELECT t2.AlbumId FROM tracks t2
                                      WHERE t2.TrackId = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,410,0.9951456310679612
yes,2,0.0048543689320388345
